In [21]:
import pandas as pd
akshit_df = './mlchallenge_set_2021.tsv'
akshit_valid = './mlchallenge_set_validation.tsv'
sam_df = 'C:/Users/sjmal/OneDrive/Desktop/ML/2021/mlchallenge_set_2021_edited.txt'
sam_valid = 'C:/Users/sjmal/OneDrive/Desktop/ML/2021/mlchallenge_set_validation.tsv'
SA_valid=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_validation.tsv',header=None)
SA_df=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_2021.tsv',header=None)
df=SA_df
valid=SA_valid
#df = pd.read_table(akshit_df)
#valid = pd.read_table(akshit_valid,sep='\t')
df.columns=['category','primary_image_url','All Links','Tags','index']
valid.columns=['ID', 'Group']

In [22]:
#create dictionary of groups with list of IDs in that group
dict = {}
total=0
for index,row in valid.iterrows():
    if row['Group'] in dict:
        dict[row['Group']].append(row['ID'])
    else:
        dict[row['Group']] = [row['ID']]
#list of groups with multiple IDs
matches = []
for key,value in dict.items():
    if len(value) > 1:
        total+=len(value)
        matches.append(key)
print(len(matches))
print(total)

990
2480


In [23]:
print(df.head())
print(valid.head())

   category                                  primary_image_url  \
0         2  https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYYA...   
1         2  https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIA...   
2         2  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIA...   
3         2  http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAA...   
4         2  https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scsA...   

                                           All Links  \
0  https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYYA...   
1  https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIA...   
2  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIA...   
3  http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAA...   
4  https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scsA...   

                                                Tags  index  
0  (Brand:Shimano,US Shoe Size (Men's):4.5,Modifi...      0  
1  (Color:Gray/White,Country/Region of Manufactur...      1  
2  (Style:Cleats,Color:White Orange,US Shoe Size ...      2  
3  (Width:Medium (

In [24]:
#split links into list (for now only does first 100)
i = 0
for link in df['All Links'][0:100]:
    df['All Links'][i] = link.split(';')
    i+=1
print(df['All Links'][0:100])

0     [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYY...
1     [https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPI...
2     [https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flI...
3     [http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcA...
4     [https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scs...
                            ...                        
95    [https://i.ebayimg.com/00/s/MTU5OVgxNTgx/z/vMA...
96    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/GoA...
97    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/Hm0...
98    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/VhA...
99    [https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/n4o...
Name: All Links, Length: 100, dtype: object


In [25]:
import re
def register_attributes(attribute, all_attributes):
    attribute = re.sub(r'[()]','', attribute)
    attribute = re.split(r',', attribute)
    attribute = [a.split(':') for a in attribute]
    for i, a in enumerate(attribute):
        attribute[i] = [s.strip() for s in a]
        all_attributes.add(attribute[i][0])
    #print(f'atttribute is: {attribute}')
    mapping = {}
    #for i in range(len(attribute) - 1):
    #    if i == len(attribute) - 2:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:]
    #    else:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:-1]
    return(attribute)

def map_attributes(attribute, num_attributes, index_to_attr):
    attribute = re.sub(r'[()]','', attribute)
    attribute = re.split(r',', attribute)
    attribute = [a.split(':') for a in attribute]
    all_attributes_for_row = [None] * num_attributes
    for i, a in enumerate(attribute):
        attribute[i] = [s.strip() for s in a]
        #print(f'index: {attr_to_index[attribute[i][0]]}')
        if len(attribute[i]) > 1:
            all_attributes_for_row[attr_to_index[attribute[i][0]]] = attribute[i][1]
    mapping = {}
    #for i in range(len(attribute) - 1):
    #    if i == len(attribute) - 2:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:]
    #    else:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:-1]
    return all_attributes_for_row
m = 2000
all_attributes = set()
all_maps = []
for index,row in df[0:m].iterrows():
    register_attributes(row['Tags'], all_attributes)

all_attributes = list(all_attributes)
attr_to_index = {all_attributes[i]: i for i in range(len(all_attributes))}
#print(attr_to_index)
#print(f'numAttributes: {len(all_attributes)}')

for index,row in df[0:m].iterrows():
    all_maps.append(map_attributes(row['Tags'], len(all_attributes), attr_to_index))
#print(all_maps)


In [27]:
#dataframe with attribute values
categories = pd.DataFrame(all_maps)
print(categories)

       0     1     2     3     4     5     6      7     8     9    ...   309  \
0     None  None  None  None  None  None  None   None  None  None  ...  None   
1     None  None  None  None  None  None  None   None  None  None  ...  None   
2     None  None  None  None  None  None  None   None  None  None  ...  None   
3     None  None  None  None  None  None  None   None  None  None  ...  None   
4     None  None  None  None  None  None  None   None  None  None  ...  None   
...    ...   ...   ...   ...   ...   ...   ...    ...   ...   ...  ...   ...   
1995  None  None  None  None  None  None  None   None  None  None  ...  None   
1996  None  None  None  None  None  None  None   None  None  None  ...  None   
1997  None  None  None  None  None  None  None   None  None  None  ...  None   
1998  None  None  None  None  None  None  None  52642  None  None  ...  None   
1999  None  None  None  None  None  None  None  65659  None  None  ...  None   

       310   311   312   313   314   31

In [19]:
from sklearn.preprocessing import OneHotEncoder
x = OneHotEncoder().fit_transform(categories)
print(x)
print(type(x))

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['NoneType', 'str']

In [92]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(compute_distances=True,compute_full_tree = True,distance_threshold = 1.5,n_clusters=None).fit(x.todense())
print(clustering)

AgglomerativeClustering(compute_distances=True, compute_full_tree=True,
                        distance_threshold=1.5, n_clusters=None)


In [93]:
print("LABELS")
print(clustering.labels_)
print(len(clustering.labels_))
print(len(set(clustering.labels_)))
print("DISTAnCES")
print(clustering.distances_)
print("num connected components")
print(clustering.n_connected_components_)
# make this better
# make this work on the entire dataset
# fix nonetypes
# don't punish missing attributes, but punish conflicts. how do we encode this?

LABELS
[ 921 1653 1173 ...  572  285  170]
2000
1765
DISTAnCES
[ 0.          0.          0.         ... 34.87119155 43.63236752
 67.88776103]
num connected components
1


In [94]:
groups = {}
for i, label in enumerate(clustering.labels_):
    if label not in groups:
        groups[label] = []
    groups[label].append(i)
groups = {label: groups[label] for label in groups if len(groups[label]) > 1}
print(groups)
for label in groups:
    print(f'GROUP: {label}')
    for item in groups[label]:
        print(df['Tags'][item])
    print('-----------------')


{44: [6, 34], 37: [15, 26, 156, 179, 519], 387: [16, 52], 101: [17, 1543], 777: [21, 1540], 38: [23, 1521], 158: [25, 1682], 92: [27, 231, 1549], 348: [31, 1668], 127: [33, 1738], 132: [37, 44, 1627], 331: [38, 46], 56: [40, 88], 186: [48, 97, 166, 1585], 49: [56, 1553], 93: [60, 183, 1739], 139: [68, 248], 167: [75, 1611], 70: [77, 169], 57: [78, 85], 58: [79, 136, 1554], 83: [80, 91, 765, 1523, 1594], 64: [87, 1792], 32: [96, 990], 65: [104, 195, 1602, 1633, 1643], 193: [108, 1903], 30: [124, 200], 136: [126, 1994], 165: [141, 719], 31: [148, 1581], 51: [151, 1607], 104: [168, 313], 53: [178, 506, 1800], 45: [180, 834], 102: [186, 1593], 378: [188, 1625], 772: [204, 244, 267, 308, 485, 923, 1276, 1374, 1517, 1844, 1870, 1887], 80: [222, 246, 344, 632], 96: [230, 1295, 1573], 24: [240, 299], 173: [296, 1671], 39: [321, 1916], 77: [366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 3

In [98]:
%matplotlib qt
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
import numpy as np
from PIL import Image
import requests
from io import BytesIO

def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    #response = requests.get(url)
    #img = Image.open(BytesIO(response.content))
    link_labels = [df['Tags'][i] for i in clustering.labels_]
    dendrogram(linkage_matrix, labels = link_labels)
plot_dendrogram(clustering)
plt.show()

In [31]:
import re
from collections import Counter
freq=Counter()
attribute=[['']]*len(df)
trialrange=100
for x in range(trialrange):#range(int(len(df)/10)):#len(df)
    attribute[x]=df.iloc[x,3].lower()
    attribute[x] = re.sub(r'[()]','', attribute[x])
    attribute[x] = re.split(r',', attribute[x])
    attribute[x] = [a.split(':') for a in attribute[x]]
    freq+=Counter([i[0] for i in attribute[x]])
    tempdict={}
    for i in attribute[x]:

            try:
                tempdict[i[0]]=float(i[1])
            except:
                try:
                    tempdict[i[0]]=i[1]
                except:
                    pass
    attribute[x]=tempdict

df['seg']=attribute
#print(df['seg'])

In [36]:
Brands=[]
Images=[]
print(trialrange)
for i in range(trialrange):
    try:
        #df['seg'].iloc[i]['brand']
        #df['primary_image_url'].iloc[i]
        #print(df['seg'].iloc[i]['brand'])
        #print(df['primary_image_url'].iloc[i])
        Brands.append(df['seg'].iloc[i]['brand'])
        Images.append(df['primary_image_url'].iloc[i])
    except:
        print('not possible at: ',i)

100
